In [1]:
import os
os.chdir('C:\\Users\\HP\\Desktop\\Akaike\\Basics')
os.getcwd()

'C:\\Users\\HP\\Desktop\\Akaike\\Basics'

In [2]:
import pandas as pd
import json
from dotenv import load_dotenv
import re
import copy
import plotly.graph_objects as go
from langchain_community.chat_models import  ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
load_dotenv(".env")


True

## automatic metrics part 

In [3]:
#  def metrics_for_table_prompt( df_head: str, df_shape: str = '', df_nunique: str = '',
#                                  column_description: str = ""):
#         prompt = '''
# Consider yourself as a business Analyst and user as a business stakeholder.
# You will be given, head(1st 5 rows) of the dataset, no. of unique values in the each column.
# You are tasked to do the following:
# - Understand the type of data given and basis that generate useful metrics for the business stakeholder strictly based on given data
# - The metrics you come up with can be of the following categories:
#     - Aggregates: Single number aggreagates based the dataset. For eg: Total Profit, Average order value, etc.
#     - Time Series: In case, data contains some time series, give metrics which highlight trends over time. For Example: category wise sales month on month, average closing value every quarter  etc.
#     - Group Aggregation: In case of categorical data, analysis based on that value can also be useful. For eg: Average order value per Product category, Age Group wise sales Analysis. Profit Across gender
#     - Combination: Combination of Time Series and Group Aggregation
# - Make sure Metrics are useful to the stakeholder and not gibberish
# Example for Reference:
# Head of the dataset:
# 'Order_Date,Time,Aging,Customer_Id,Gender,Device_Type,Customer_Login_type,Product_Category,Product,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Payment_method\n2018-01-02,10:56:33,8.0,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1.0,0.3,46.0,4.6,Medium,credit_card\n2018-07-24,20:41:37,2.0,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1.0,0.3,112.0,11.2,Medium,credit_card\n2018-11-08,08:38:49,8.0,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5.0,0.1,31.2,3.1,Critical,credit_card\n2018-04-18,19:28:06,7.0,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1.0,0.3,26.2,2.6,High,credit_card\n2018-08-13,21:18:39,9.0,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1.0,0.3,160.0,16.0,Critical,credit_card\n'
# shape of the dataset: (5,16)
# No. of unique values per column:
# 'Order_Date 356\nTime 35275\nAging 11\nCustomer_Id38997\nGender 2\nDevice_Type2\nCustomer_Login_type4\nProduct_Category 4\nProduct 42\nSales 39\nQuantity 5\nDiscount 5\nProfit 391\nShipping_Cost153\nOrder_Priority 4\nPayment_method 5'

# Thought: This is a sales dataset for my stakeholder, metrics related to Average Profit(Aggregates), Average Sales(Aggregates), Average Shipping_Cost(Aggregates),Month on Month Trends of Sales/profit/Shipping cost(Time Series), this can be best visualized through line charts,Sales/Profit/Average order value Across payment method/Device_Type/Order_Priority(Group Aggregation), these can be best visualized through either bar plots of grouped bar plots or pie charts or stacked chart based on number of unique values.Trend of Sales across males and females over time (combination), this can be analyzed threough multiline line charts
# Result :{{"aggregates":["Average Profit", "Average Sales", "Average Shipping_Cost"],"time_series":[{{"metric": "Month on month sales_trend","chart_type":"line",groupby=None}},...],"group_aggregation":[{{"metric":"Revenue per payment method","chart_type":"bar"}}],"combination"[{{"metric":"Sales across gender over time","chart_type":"multiline"}}]}}

# Important Points to Note:
# - All datasets might not have all the 3 types of metrics, leave it as blank list in such cases
# - Maximum possible number of unique values for the column to be considered for group aggregates is 8
# - Chart types given in thought are only for reference, you may give other chart types which could help visualize metric better based on number of unique values and shape of the dataset,like box plot, pie chartetc.
# - possible chart types include ['line', 'bar', 'area', 'histogram', 'pie', 'stacked_bar', 'multiline'(combination of group aggregate and time series), 'grouped_bar']
# - Different metrics of a Metric type can have different visualizations
# - Give metrics name in a professional and formal manner, remember a business stakeholder has to see it.
# - Give atmost 4 metrics for each catgory
# - Use the same structure as given in the above example, JSON keys should exactly be as mentioned
# - Make sure that the output is json serizable, need to directly use json.loads on the output
# Here is what you have:
# Head of the dataset:
# {df_head}
# column description:{column_description}
# shape of the dataset: {df_shape}
# No. of unique values per column:
# {df_nunique}
# Result:
#         '''
#         return prompt.format(df_head=df_head, column_description=column_description, df_shape=df_shape,
#                                      df_nunique=df_nunique)


In [4]:
# def metrics_for_table_prompt(df_head: str, df_info: str, df_shape: str = '', df_nunique: str = '', column_description: str = ""):
#     prompt = '''
#     Consider yourself as a business analyst and the user as a business stakeholder.
#     You will be given the head (1st 5 rows) of the dataset, the number of unique values in each column.
#     You are tasked to do the following:
#     - Understand the type of data given and based on that generate useful metrics for the business stakeholder strictly based on the given data.
#     - The metrics you come up with can be of the following categories:
#         - Aggregates: Single number aggregates based on the dataset. For example: Total Profit, Average order value, etc.
#         - Time Series: In case the data contains some time series, give metrics which highlight trends over time. For example: category-wise sales month on month, average closing value every quarter, etc.
#         - Group Aggregation: In case of categorical data, analysis based on that value can also be useful. For example: Average order value per Product category, Age Group wise sales Analysis, Profit Across gender, etc.
#         - Combination: Combination of Time Series and Group Aggregation.
#     - Make sure Metrics are useful to the stakeholder and not gibberish.
#      Example for Reference:
#     Head of the dataset:
#     'Order_Date,Time,Aging,Customer_Id,Gender,Device_Type,Customer_Login_type,Product_Category,Product,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Payment_method\n2018-01-02,10:56:33,8.0,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1.0,0.3,46.0,4.6,Medium,credit_card\n2018-07-24,20:41:37,2.0,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1.0,0.3,112.0,11.2,Medium,credit_card\n2018-11-08,08:38:49,8.0,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5.0,0.1,31.2,3.1,Critical,credit_card\n2018-04-18,19:28:06,7.0,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1.0,0.3,26.2,2.6,High,credit_card\n2018-08-13,21:18:39,9.0,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1.0,0.3,160.0,16.0,Critical,credit_card\n'
#     shape of the dataset: (51290,16)
#     No. of unique values per column:
#     'Order_Date 356\nTime 35275\nAging 11\nCustomer_Id38997\nGender 2\nDevice_Type2\nCustomer_Login_type4\nProduct_Category 4\nProduct 42\nSales 39\nQuantity 5\nDiscount 5\nProfit 391\nShipping_Cost153\nOrder_Priority 4\nPayment_method 5'
#     Info of the dataset:
#     'Order_Date,51290,non-null,object,Time,51290,non-null,object,Aging,51289,non-null,float64,Customer_Id,51290,non-null,int64,Gender,51290,non-null,object,Device_Type,51290,non-null,object,Customer_Login_type,51290,non-null,object,Product_Category,51290,non-null,object,Product,51290,non-null,object,Sales,51289,non-null,float64,Quantity,51288,non-null,float64,Discount,51289,non-null,float64,Profit,51290,non-null,float64,Shipping_Cost,51289,non-null,float64,Order_Priority,51288,non-null,object,Payment_method,51290,non-null,object'
    
#     Thought: This is a sales dataset for my stakeholder, metrics related to Average Profit(Aggregates), Average Sales(Aggregates), Average Shipping_Cost(Aggregates),Month on Month Trends of Sales/profit/Shipping cost(Time Series), this can be best visualized through line charts,Sales/Profit/Average order value Across payment method/Device_Type/Order_Priority(Group Aggregation), these can be best visualized through either bar plots of grouped bar plots or pie charts or stacked chart based on number of unique values.Trend of Sales across males and females over time (combination), this can be analyzed threough multiline line charts'
#     Result :{{"aggregates":["Average Profit", "Average Sales", "Average Shipping_Cost"],"time_series":[{{"metric": "Month on month sales_trend","chart_type":"line",groupby=None}},...],"group_aggregation":[{{"metric":"Revenue per payment method","chart_type":"bar"}}],"combination"[{{"metric":"Sales across gender over time","chart_type":"multiline"}}]}}


#     Important Points to Note:
#     - All datasets might not have all the 3 types of metrics, leave it as a blank list in such cases.
#     - Maximum possible number of unique values for the column to be considered for group aggregates is 8.
#     - Chart types given in thought are only for reference; you may give other chart types which could help visualize metrics better based on the number of unique values and the shape of the dataset, like box plot, pie chart, etc.
#     - Possible chart types include ['line', 'bar', 'area', 'histogram', 'pie', 'stacked_bar', 'multiline' (combination of group aggregate and time series), 'grouped_bar'].
#     - Different metrics of a Metric type can have different visualizations.
#     - Give metrics names in a professional and formal manner; remember a business stakeholder has to see it.
#     - Give at most 4 metrics for each category.
#     - Avoid redundancy in metrics; choose the most informative and unique metrics.
#     - Use the same structure as given in the above example; JSON keys should be exactly as mentioned.
#     - Make sure that the output is JSON serializable; need to directly use json.loads on the output.
#     Here is what you have:
#     Head of the dataset:
#     {df_head}
#     Column description: {column_description}
#     Shape of the dataset: {df_shape}
#     No. of unique values per column:
#     {df_nunique}
#     Info of the dataset:
#     {df_info}
#     Result:
#     '''
#     return prompt.format(df_head=df_head, column_description=column_description, df_shape=df_shape,
#                          df_nunique=df_nunique, df_info=df_info)


In [5]:
def metrics_for_table_prompt(df_head: str, df_info: str, df_shape: str = '', df_nunique: str = '', column_description: str = ""):
    prompt = '''
    Consider yourself as a business analyst and the user as a business stakeholder.
    You will be given the head (1st 5 rows) of the dataset, the number of unique values in each column.
    You are tasked to do the following:
    - Understand the type of data given and based on that generate useful metrics for the business stakeholder strictly based on the given data.
    - The metrics you come up with can be of the following categories:
        - Aggregates: Single number aggregates based on the dataset. For example: Total Profit, Average order value, etc.
        - Time Series: In case the data contains some time series, give metrics which highlight trends over time. For example: category-wise sales month on month, average closing value every quarter, etc.
        - Group Aggregation: In case of categorical data, analysis based on that value can also be useful. For example: Average order value per Product category, Age Group wise sales Analysis, Profit Across gender, etc.
        - Combination: Combination of Time Series and Group Aggregation.
    - Make sure Metrics are useful to the stakeholder and not gibberish.
     Example for Reference:
     Head of the dataset:
    'Order_Date,Time,Aging,Customer_Id,Gender,Device_Type,Customer_Login_type,Product_Category,Product,Sales,Quantity,Discount,Profit,Shipping_Cost,Order_Priority,Payment_method\n2018-01-02,10:56:33,8.0,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1.0,0.3,46.0,4.6,Medium,credit_card\n2018-07-24,20:41:37,2.0,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1.0,0.3,112.0,11.2,Medium,credit_card\n2018-11-08,08:38:49,8.0,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5.0,0.1,31.2,3.1,Critical,credit_card\n2018-04-18,19:28:06,7.0,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1.0,0.3,26.2,2.6,High,credit_card\n2018-08-13,21:18:39,9.0,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1.0,0.3,160.0,16.0,Critical,credit_card\n'
    shape of the dataset: (51290,16)
    No. of unique values per column:
    'Order_Date 356\nTime 35275\nAging 11\nCustomer_Id38997\nGender 2\nDevice_Type2\nCustomer_Login_type4\nProduct_Category 4\nProduct 42\nSales 39\nQuantity 5\nDiscount 5\nProfit 391\nShipping_Cost153\nOrder_Priority 4\nPayment_method 5'
    Info of the dataset:
    'Order_Date,51290,non-null,object,Time,51290,non-null,object,Aging,51289,non-null,float64,Customer_Id,51290,non-null,int64,Gender,51290,non-null,object,Device_Type,51290,non-null,object,Customer_Login_type,51290,non-null,object,Product_Category,51290,non-null,object,Product,51290,non-null,object,Sales,51289,non-null,float64,Quantity,51288,non-null,float64,Discount,51289,non-null,float64,Profit,51290,non-null,float64,Shipping_Cost,51289,non-null,float64,Order_Priority,51288,non-null,object,Payment_method,51290,non-null,object'
    Result:
    {{
        "aggregates": ["Average Profit", "Average Sales", "Average Shipping Cost"],
        "time_series": [{{"metric": "Month-on-Month Sales Trend", "chart_type": "line", "groupby": "Order_Date"}},{{"metric": "Yearly Shipping Cost Trend", "chart_type": "line", "groupby": "Order_Date"}}],
        "group_aggregation": [{{"metric": "Average Order Value per Product Category", "chart_type": "bar", "groupby": "Product_Category"}},{{"metric": "Profit Distribution Across Gender", "chart_type": "pie", "groupby": "Gender"}},{{"metric": "Sales Distribution Across Device Type", "chart_type": "bar", "groupby": "Device_Type"}}],
        "combination": [{{"metric": "Monthly Sales Trend Across Gender", "chart_type": "multiline", "groupby": "Order_Date"}},{{"metric": "Quarterly Profit Trend Across Product Category", "chart_type": "multiline", "groupby": "Product_Category"}}]
    }}
     Important Points to Note:
    - All datasets might not have all the 3 types of metrics, leave it as a blank list in such cases.
    - Maximum possible number of unique values for the column to be considered for group aggregates is 8.
    - Chart types given in thought are only for reference; you may give other chart types which could help visualize metrics better based on the number of unique values and the shape of the dataset, like box plot, pie chart, etc.
    - Possible chart types include ['line', 'bar', 'area', 'histogram', 'pie', 'stacked_bar', 'multiline' (combination of group aggregate and time series), 'grouped_bar'].
    - Different metrics of a Metric type can have different visualizations.
    - Give metrics names in a professional and formal manner; remember a business stakeholder has to see it.
    - Give at most 4 metrics for each category.
    - Avoid redundancy in metrics; choose the most informative and unique metrics.
    - Use the same structure as given in the above example; JSON keys should be exactly as mentioned.
    - Make sure that the output is JSON serializable; need to directly use json.loads on the output.
    Here is what you have:
    Head of the dataset:
    {df_head}
    Column description: {column_description}
    Shape of the dataset: {df_shape}
    No. of unique values per column:
    {df_nunique}
    Info of the dataset:
    {df_info}
    Result:
    '''

    return prompt.format(df_head=df_head, column_description=column_description, df_shape=df_shape,
                         df_nunique=df_nunique, df_info=df_info)


In [27]:
hmvi = pd.read_csv("../insurance_data.csv")

In [28]:
hmvi.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000093009,MIDDLE ATLANTIC SPORTS CO INC,UNION,NJ,7083,WELLS FARGO BANK NATL ASSOC,SD,0,2-Jun-80,1980,...,N,N,24-Jun-91,22-Jul-80,"$600,000.00",$0.00,CHGOFF,"$208,959.00","$600,000.00","$499,998.00"
1,1000554001,PAUL E. & JUDY A. FAATZ,KINSMAN,OH,44428,CORTLAND SAVINGS & BK. CO.,OH,0,28-Feb-97,1997,...,N,Y,18-Apr-02,30-Jun-97,"$47,000.00",$0.00,CHGOFF,"$14,084.00","$47,000.00","$37,600.00"
2,1000653000,LARRY SCHOETTMER FORD INC,EDINBURGH,IN,46124,JPMORGAN CHASE BANK NATL ASSOC,IN,0,11-Jun-80,1980,...,Y,N,4-Oct-89,31-Jul-80,"$197,485.00",$0.00,CHGOFF,"$44,374.00","$200,000.00","$150,000.00"
3,1000726007,NEW DIMENSION HOMES INC,CLINTON (CENSUS NAME FOR CLINT,ME,4927,WELLS FARGO BANK NATL ASSOC,SD,236115,7-Feb-06,2006,...,0,N,26-Jun-14,28-Feb-06,"$1,350,000.00",$0.00,CHGOFF,"$1,043,508.00","$1,350,000.00","$1,012,500.00"
4,1000735003,Mykols Marine Inc.,Spring Lake,MI,49417,WELLS FARGO BANK NATL ASSOC,SD,713930,31-Mar-97,1997,...,0,N,4-Oct-05,28-Feb-98,"$25,000.00",$0.00,CHGOFF,"$23,484.00","$25,000.00","$12,500.00"


In [29]:
def model_response(query:str):
    chat = ChatOpenAI(model="gpt-4o",
                    temperature = 0,
                    model_kwargs= {
                    "response_format": {"type": "json_object"},
                    "seed" : 42})

    messages = [
        SystemMessage(
            content="You are a business analyst designed to give key insights,designed to output json"),
        HumanMessage(
            content=query),
    ]
    return json.loads(chat(messages).content)

In [30]:
df_head = hmvi.head().to_csv(index=False)

In [31]:
df_shape = hmvi.shape

In [32]:
df_nunique =hmvi.nunique()

In [33]:
df_info=hmvi.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25243 entries, 0 to 25242
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   LoanNr_ChkDgt      25243 non-null  int64 
 1   Name               25243 non-null  object
 2   City               25243 non-null  object
 3   State              25243 non-null  object
 4   Zip                25243 non-null  int64 
 5   Bank               25243 non-null  object
 6   BankState          25243 non-null  object
 7   NAICS              25243 non-null  int64 
 8   ApprovalDate       25243 non-null  object
 9   ApprovalFY         25243 non-null  int64 
 10  Term               25243 non-null  int64 
 11  NoEmp              25243 non-null  int64 
 12  NewExist           25243 non-null  int64 
 13  CreateJob          25243 non-null  int64 
 14  RetainedJob        25243 non-null  int64 
 15  FranchiseCode      25243 non-null  int64 
 16  UrbanRural         25243 non-null  int64

In [34]:
df_nunique = str(df_nunique).replace("  "," ")
df_nunique

'LoanNr_ChkDgt    25243\nName         24342\nCity         6369\nState          51\nZip          9443\nBank         1004\nBankState        52\nNAICS         909\nApprovalDate     1548\nApprovalFY       18\nTerm          304\nNoEmp         121\nNewExist         3\nCreateJob        49\nRetainedJob       90\nFranchiseCode     315\nUrbanRural        3\nRevLineCr        4\nLowDoc          3\nChgOffDate      3709\nDisbursementDate   1062\nDisbursementGross   9840\nBalanceGross       1\nMIS_Status        2\nChgOffPrinGr     19200\nGrAppv        1454\nSBA_Appv       1930\ndtype: int64'

## prompt output

In [35]:
dfmetr = model_response(metrics_for_table_prompt(df_head=df_head,df_info=df_info,df_nunique=df_nunique,df_shape = df_shape))


In [36]:
dfmetr

{'aggregates': ['Average Disbursement Gross',
  'Total Number of Loans',
  'Average Term of Loans',
  'Total Number of Jobs Created'],
 'time_series': [{'metric': 'Yearly Loan Approval Trend',
   'chart_type': 'line',
   'groupby': 'ApprovalFY'},
  {'metric': 'Yearly Disbursement Gross Trend',
   'chart_type': 'line',
   'groupby': 'ApprovalFY'}],
 'group_aggregation': [{'metric': 'Average Disbursement Gross per State',
   'chart_type': 'bar',
   'groupby': 'State'},
  {'metric': 'Loan Distribution Across Bank States',
   'chart_type': 'pie',
   'groupby': 'BankState'},
  {'metric': 'Average Number of Employees per NAICS Category',
   'chart_type': 'bar',
   'groupby': 'NAICS'},
  {'metric': 'Loan Status Distribution',
   'chart_type': 'pie',
   'groupby': 'MIS_Status'}],
 'combination': [{'metric': 'Yearly Loan Approval Trend Across States',
   'chart_type': 'multiline',
   'groupby': 'State'},
  {'metric': 'Quarterly Disbursement Gross Trend Across NAICS Categories',
   'chart_type':

## deliverables

- optimize the prompt
- check if the metrics generated are correct or not
- check the chart type
- see how it can be connected wit custom metrics